In [52]:
# 신규 게시물 DB 저장 및 형태소 분석해서 DB에 저장
# ++ 만료된 게시물 is_new = 0 으로 업데이트 코드 추가할 것 
# 1시간마다 실행할 것

from bs4 import BeautifulSoup
from selenium import webdriver  #chrome webdriver 다운로드 필요
from konlpy.tag import Komoran
import time
from datetime import datetime, timedelta
import re
from selenium.webdriver.common.alert import Alert
import pymysql

hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
category = {'정치개혁': 0, '외교/통일/국방': 1, '일자리': 2, '미래': 3, '성장동력': 4, '농산어촌': 5, '보건복지': 6, '육아/교육': 7, '안전/환경': 8, '저출산/고령화대책': 9, '행정': 10, '반려동물': 11, '교통/건축/국토': 12, '경제민주화': 13, '인권/성평등': 14, '문화/예술/체육/언론': 15, '기타': 16}

komoran = Komoran(max_heap_size= 1024 * 6)

# DB에서 제일 최근꺼 post_id 찾기
db = pymysql.connect(host='34.64.192.134', port=3306, user='root', passwd='998', db='sys', charset='utf8', autocommit=True)
cursor = db.cursor()
sql2 = "SELECT MAX(post_id) FROM post;"
cursor.execute(sql2)
db_newest_post_id = cursor.fetchone()[0]
db.close()

# options = webdriver.ChromeOptions()
# options.add_argument("--headless")
# options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome("C:/Users/chlxo/Desktop/chromedriver")
#driver = webdriver.Chrome(executable_path='./chromedriver', options=options)
#################################################################
#제일 최신의 만료된 게시물 번호 찾기!
first_old_url = 'https://www1.president.go.kr/petitions/?c=0&only=2&page=1&order=1'
first_old_postid = "0"
while(True):
    try:
        driver.get(first_old_url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        href = soup.select('.petition_list > li > div > div > a')[0]
        first_old_postid = re.sub('/petitions/','',href.get('href'))
        break
    except IndexError:
        print("zzz")
        time.sleep(1)
newest_exired_postid = int(first_old_postid)
db = pymysql.connect(host='34.64.192.134', port=3306, user='root', passwd='998', db='sys', charset='utf8', autocommit=True)
cursor = db.cursor()
sql1 = "UPDATE post SET is_new = 0 WHERE post_id <= %s and is_new = 1"
cursor.execute(sql1, (newest_exired_postid))
post_rows = cursor.fetchall()
#################################################################
driver.quit()

driver = webdriver.Chrome("C:/Users/chlxo/Desktop/chromedriver")
# 제일 최신의 진행중인 게시물 번호 찾기!
first_new_url = 'https://www1.president.go.kr/petitions/?c=0&only=1&page=1&order=1'

most_recent_number = "0"
while(True):
    try:
        driver.get(first_new_url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        href = soup.select('.petition_list > li > div > div > a')[0]
        most_recent_number = re.sub('/petitions/','',href.get('href'))
        break
    except IndexError:
        print("zzz")
        time.sleep(1)              
count = int(most_recent_number)
driver.quit()

driver = webdriver.Chrome("C:/Users/chlxo/Desktop/chromedriver")
url = 'https://www1.president.go.kr/petitions/{}'

while db_newest_post_id < count:
    link = url.format(count)
    driver.get(link)  

    html = driver.page_source 
    soup = BeautifulSoup( html, 'html.parser' )
    try:
        post_id = count
        category_name = soup.select('#cont_view > div > div > div > div > div > div > ul.petitionsView_info_list > li')[0]
        category_id = category[category_name.text[4:]]
        start = soup.select('#cont_view > div > div > div > div > div > div > ul.petitionsView_info_list > li')[1]
        start_date = start.text[4:]
        end = soup.select('#cont_view > div > div > div > div > div > div > ul.petitionsView_info_list > li')[2]
        end_date = end.text[4:]
        title = soup.select_one('#cont_view > div > div > div > div > div > h3.petitionsView_title').get_text()
        title = hangul.sub('', title)
        agree = soup.select_one('#cont_view > div > div > div > div > div > h2 > span').get_text()
        agree_count = agree.replace(',', '')
            
        # 신규 게시물 post table에 저장    
        db = pymysql.connect(host='34.64.192.134', port=3306, user='root', passwd='998', db='sys', charset='utf8', autocommit=True)
        cursor = db.cursor()
       
        input_data = (post_id, category_id, title, agree_count, start_date, end_date, True)
        sql = '''
                INSERT INTO post
                (post_id, category_id, title, agree_count, start_date, end_date, is_new)
                VALUES(%s, %s, %s, %s, %s, %s, %s);
                '''
        
        cursor.execute(sql, input_data)
        db.close()        
        
        # 신규 게시물 형태소 분석
        db = pymysql.connect(host='34.64.192.134', port=3306, user='root', passwd='998', db='sys', charset='utf8', autocommit=True)
        cursor = db.cursor()
        
        for keyword in set(s for s in komoran.nouns(title) if len(s) > 1):
            sql2 = "INSERT INTO post_keyword(post_id, keyword) VALUES(%s, %s);"
            cursor.execute(sql2, (post_id, keyword))
        
        db.close()
        
    except IndexError:
        None
#         print(post_id, " : 비공개된 청원 or 게시물 없음")
   
    count-=1
    time.sleep(3)

driver.quit()

driver = webdriver.Chrome("C:/Users/chlxo/Desktop/chromedriver")
# 신규 게시물 형태소 분석
db = pymysql.connect(host='34.64.192.134', port=3306, user='root', passwd='998', db='sys', charset='utf8', autocommit=True)
cursor = db.cursor()
sql2 = "SELECT post_id, agree_count FROM post WHERE is_new=1;"
cursor.execute(sql2)
new_post_id_agree_counts = cursor.fetchall()
db.close()

collect_time = datetime.now() + timedelta(hours=9)

for new_post_id_agree_count in new_post_id_agree_counts:
    post_id = new_post_id_agree_count[0]
    link = url.format(post_id)
    driver.get(link)
    html = driver.page_source 
    soup = BeautifulSoup( html, 'html.parser' )
    agree = soup.select_one('#cont_view > div > div > div > div > div > h2 > span').get_text()
    agree_count = agree.replace(',', '')
    
    db = pymysql.connect(host='34.64.192.134', port=3306, user='root', passwd='998', db='sys', charset='utf8', autocommit=True)
    cursor = db.cursor()
    sql2 = "UPDATE post SET agree_count = %s WHERE post_id = %s;"
    cursor.execute(sql2, (agree_count, post_id))
    
    agree_diff = int(agree_count) - new_post_id_agree_count[1]
    sql2 = '''
    INSERT INTO post_agree_increment VALUES(%s, %s, %s)
    ON DUPLICATE KEY UPDATE collect_time = %s, agree_increment = %s;
    '''
    cursor.execute(sql2, (post_id, collect_time, agree_diff, collect_time, agree_diff))
    
    db.close()
    time.sleep(3)
    
driver.quit()

KeyboardInterrupt: 

In [11]:
import pymysql
import datetime
from collections import defaultdict

now = datetime.datetime.now() + datetime.timedelta(hours=9)
print(now)

db = pymysql.connect(host='34.64.192.134', port=3306, user='root', passwd='998', db='sys', charset='utf8', autocommit=True)
cursor = db.cursor()
for i in range(0, 17):
    dict_agree_sum = defaultdict(int)
    dict_post_sum = defaultdict(int)
    
    dict_new_agree_sum = defaultdict(int)
    dict_new_post_sum = defaultdict(int)
    
    sql = "SELECT start_date, agree_sum, post_sum FROM category_post_count WHERE category_id = %s;"
    cursor.execute(sql, (i))
    category_rows = cursor.fetchall()
    
    for category_row in category_rows :
        dict_agree_sum[category_row[0]]+=category_row[1]
        dict_post_sum[category_row[0]]+=category_row[2]
        
    sql = "SELECT start_date, agree_count FROM post WHERE is_new = 1 and category_id = %s;"
    cursor.execute(sql, (i))
    post_rows = cursor.fetchall()
        
    for post_row in post_rows :
        dict_new_agree_sum[post_row[0]]+=post_row[1]
        dict_new_post_sum[post_row[0]]+=1
    
    sql = "SELECT start_date, agree_count FROM post WHERE is_new = 1 and category_id = %s;"
    cursor.execute(sql, (i))

    agree_increment = 0
    post_increment = 0
    
    for key, value in dict_new_agree_sum.items():
        sql = "REPLACE INTO category_post_count VALUES(%s, %s, %s, %s);"
        cursor.execute(sql, (i, key, value, dict_new_post_sum[key]))
        
        agree_increment = value - dict_agree_sum[key]
        post_increment = dict_new_post_sum[key] - dict_post_sum[key]
        
    sql = "INSERT INTO increment_of_category VALUES (%s, %s, %s, %s);"
    cursor.execute(sql, (i, now, agree_increment, post_increment))
    
    sql = "INSERT INTO keyword_ranking_of_category VALUES (%s, %s, %s);"
    cursor.execute(sql, (i, now, agree_increment + post_increment))
            
db.close()

2020-10-18 11:59:01.284123


In [16]:
from gensim.summarization.summarizer import summarize
s = '''
저는 김포에서 송도로 자주 출퇴근하면서 제2외곽순환도로를 자주 이용하는 사람입니다.
인천방향 청라원창 톨게이트에는 고속도로 한복판임에도 불구하고 하이패스톨게이트가 하나뿐입니다. 이년전엔 예산문제라고 하더니 왜 양촌 검단에는 두개가 생겼냐고 하니까 말을 못 하더라구요.
문제는 고속도로 한복판에 하이패스가 하나뿐이라 모르고 가던사람들이 끼여들면서 극심한정체, 또 엄청난 사고가 남니다. 제가 직접 본 사고만 셀수없습니다. 여기서 급브레이크 잡으면서 항상 일차선 도로개보수를 합니다.
너무 위험합니다.
살펴봐주세요.
'''
print(len(summarize(s, word_count = 10)))

0


In [ ]:
!pip uninstall lexrankr

In [31]:
from bs4 import BeautifulSoup
from selenium import webdriver  #chrome webdriver 다운로드 필요
import time
import re
from selenium.webdriver.common.alert import Alert
from datetime import datetime, timedelta

driver = webdriver.Chrome("C:/Users/chlxo/Desktop/chromedriver")
driver.implicitly_wait(3)#593660 593651 593658
driver.get("https://www1.president.go.kr/petitions/593658")   
html = driver.page_source 
soup = BeautifulSoup( html, 'html.parser' )
contents = soup.select_one('#cont_view > div > div > div > div > div > div > div.View_write').get_text()  
if(len(summarize(contents))==0):
    print(' '.join(contents[:1000].split()))
driver.close()

국민청원제도를 시행한 이유가 도대체 뭡니까?? 국민들이 힘들게 청원 동의를 해서 20만명 넘겨도, 본인들 입맛에 맞는 정해진 답변만 하는데 이 제도가 도대체 무슨 소용이 있는지 모르겠네요... 국민과 소통하겠다고 만든거 아니었나요? 아.. 소.통.만. 하겠다는 뜻으로 받아들여야 하나요?? 이런 무의미한, 세금낭비만 되는 국민청원제도 있으나 마나 한 것 같습니다. 이 글을 대통령께서 과연 읽으실까요?? 길게 쓰고 싶지도 않네요. 폐지해주세요.


In [3]:
# 신규 게시물 DB 저장 및 형태소 분석해서 DB에 저장
# ++ 만료된 게시물 is_new = 0 으로 업데이트 코드 추가할 것 
# 1시간마다 실행할 것

from bs4 import BeautifulSoup
from selenium import webdriver  #chrome webdriver 다운로드 필요
import time
import re
from selenium.webdriver.common.alert import Alert
import pymysql

category = {'정치개혁': 0, '외교/통일/국방': 1, '일자리': 2, '미래': 3, '성장동력': 4, '농산어촌': 5, '보건복지': 6, '육아/교육': 7, '안전/환경': 8, '저출산/고령화대책': 9, '행정': 10, '반려동물': 11, '교통/건축/국토': 12, '경제민주화': 13, '인권/성평등': 14, '문화/예술/체육/언론': 15, '기타': 16}

driver = webdriver.Chrome("C:/Users/chlxo/Desktop/chromedriver")
url = 'https://www1.president.go.kr/petitions/{}'
           
count = 593961
while 593488 <= count:
    link = url.format(count)
    driver.get(link)  

    html = driver.page_source 
    soup = BeautifulSoup( html, 'html.parser' )
    try:
        title = soup.select_one('#cont_view > div > div > div > div > div > h3.petitionsView_title').get_text()
        
        db = pymysql.connect(host='34.64.192.134', port=3306, user='root', passwd='998', db='sys', charset='utf8', autocommit=True)
        cursor = db.cursor()
        sql2 = "UPDATE post SET title = %s WHERE post_id = %s;"
        cursor.execute(sql2, (title, count))        
        db.close()
        
    except IndexError:
        None
   
    count-=1
    time.sleep(3)

driver.quit()